# Monthly Inflation Rates

In [114]:
import tradingeconomics as te
import pycountry
import plotly.express as px
import pandas as pd
import awoc
import numpy as np

In [132]:
te.login('jupyter:jupyter')

'You are logged in as a jupyter:jupyter'

In [118]:
my_world = awoc.AWOC()
countries = my_world.get_countries_list_of('Europe') # Europe, Africa, North America

missing_countries = {
    'Bolivia' : 'Bolivia, Plurinational State of',
    'Cape Verde' : 'Cabo Verde',
    'Cayman Islands      '  : 'Cayman Islands',   
    'Czech Republic' : 'Czechia',
    'European Union' : 'European Union',
    "Ivory Coast" : "Côte d'Ivoire",
    'Laos' : "Lao People's Democratic Republic",
    'Macau' : 'Macao',
    'Macedonia' : 'North Macedonia',
    'Moldova' : 'Moldova, Republic of',
    'Norway              '   : 'Norway',            
    'Republic of the Congo' : 'Congo, The Democratic Republic of the',
    'Russia' : 'Russian Federation',
    'Singapore           '  : 'Singapore',          
    'South Korea' : 'Korea, Republic of',
    'St Vincent and the Grenadines' : 'Saint Vincent and the Grenadines',
    'Swaziland' : 'Eswatini',
    'Sweden              ' : 'Sweden',              
    'Taiwan' : 'Taiwan, Province of China',
    'Tanzania' : 'Tanzania, United Republic of',
    'Turkey ': 'Turkey', 
    'Venezuela' : 'Venezuela, Bolivarian Republic of',
    'Vietnam' : 'Viet Nam',
    'Peru ':'Peru'
    }

In [119]:
from math import ceil
num_of_batches = ceil(sum([len(i) for i in countries]) / 200)
num_of_batches

# One API call is limited to 260 characters
batches = np.array_split(countries, num_of_batches)

In [124]:
df = pd.DataFrame()

for country  in batches:
    country_data = te.getHistoricalData(country=country, indicator='Inflation Rate', initDate='2022-06-01', output_type='df')
    
    if country_data is not None:

        count_df = country_data[['Country', 'Value']]
        df = pd.concat([df, count_df])

df

,Country,Value
0,Albania,7.40
1,Austria,8.70
2,Belarus,17.60
3,Belgium,9.65
4,Bulgaria,16.90
5,Croatia,12.10
6,Cyprus,9.60
7,Czech Republic,17.20
8,Denmark,8.20
9,Estonia,21.90


In [126]:
# add ISO names for missing countries
countries_list = df['Country'].replace(missing_countries)

In [128]:
inflation_rates = pd.DataFrame(countries_list)
inflation_rates['Inflation Rates'] = df['Value']
inflation_rates = inflation_rates.set_index('Country')
inflation_rates

,Inflation Rates
Country,
Albania,7.40
Austria,8.70
Belarus,17.60
Belgium,9.65
Bulgaria,16.90
Croatia,12.10
Cyprus,9.60
Czechia,17.20
Denmark,8.20


In [129]:
iso3_countries_names = []

for index, row in inflation_rates.iterrows():
    iso = pycountry.countries.get(name=index)
    if iso != None:
        iso3_countries_names.append(iso.alpha_3)
    else:
        iso3_countries_names.append("Nan")

In [130]:
inflation_rates['iso3'] = iso3_countries_names
inflation_rates = inflation_rates.reset_index()
inflation_rates

,Country,Inflation Rates,iso3
0,Albania,7.40,ALB
1,Austria,8.70,AUT
2,Belarus,17.60,BLR
3,Belgium,9.65,BEL
4,Bulgaria,16.90,BGR
5,Croatia,12.10,HRV
6,Cyprus,9.60,CYP
7,Czechia,17.20,CZE
8,Denmark,8.20,DNK
9,Estonia,21.90,EST


In [131]:
fig = px.choropleth(inflation_rates, locations="iso3",title= "Global Inflation Rates" ,
                        color="Inflation Rates", scope="europe", # 'world', 'usa', 'europe', 'asia', 'africa', 'north america', or 'south america'
                        hover_name="Country", # column to add to hover information
                        color_discrete_sequence=px.colors.qualitative.Dark24,
                        
                        labels={'Value':'Inflation Rate'})

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()